In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import mglearn
%matplotlib inline
import seaborn as sns
import platform
from matplotlib import font_manager , rc

if platform.system() == 'Darwin':
  rc('font' , family = 'AppleGothic')
elif platform.system() == 'Windows':
  path = 'C:/Windows/Fonts/malgun.ttf'
  font_name = font_manager.FontProperties(fname = path).get_name()
  rc('font' , family = font_name)
else:
  print('모름')
plt.rcParams['axes.unicode_minus'] = False
import warnings
warnings.filterwarnings('ignore')

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [3]:
data = wine[['alcohol','sugar','pH']]
target = wine['class']

In [4]:
from sklearn.model_selection import train_test_split

train_input , test_input , train_target , test_target = train_test_split(data , target , test_size = 0.2 , random_state = 42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1 , random_state = 42)

scores = cross_validate(rf , train_input , train_target , n_jobs = -1 , return_train_score = True)

np.mean(scores['train_score']) , np.mean(scores['test_score'])

(0.9973541965122431, 0.8905151032797809)

In [6]:
rf.fit(train_input , train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [7]:
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [8]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train_input , train_target)

LogisticRegression()

In [9]:
lr.coef_ , lr.intercept_

(array([[ 0.42688448,  0.35097153, -4.06374436]]), array([8.48811352]))

# 엑스트라 트리(Extra Tree)
 - 랜덤 포레스트와 비슷한 모델
 - 랜덤포레스트와 거의 모든 과정이 비슷하지만 , 과대적합을 억제하는 방식에 차이가 있다.
 - 부트스트랩 샘플 대신 전체 샘플을 사용
 - 여러 특성을 기준으로 랜덤하게 트리를 분할하고 이들 중 최적의 불순도를 가지는 트리를 찾아내는 방법
 - 랜덤포레스트보다 속도가 빠름

In [10]:
from sklearn.ensemble import ExtraTreesClassifier
et  = ExtraTreesClassifier(n_jobs = -1 , random_state = 42)
scores = cross_validate(et , train_input , train_target , return_train_score = True)
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9974503966084433
0.8887848893166506


# GBM(그레이디언트 부스팅) : GradientBoostingClassifier
 - 기본적으로 깊이가 3인 결정트리 100개를 사용
 - 깊이가 얕은 결정 트리를 사용하기 때문에 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음.
 - 분류에서는 로지스틱 손실함수 , 회귀에서는 평균제곱오차를 사용
 - 이전 트리의 손실 값들이 낮아지는 방향으로 회귀 결정 트리를 계속 추가하는 알고리즘
 - max_depth 또는 learning_rate(default = 0.1) 파라미터를 추가해 과대적합 억제 가능
 - 학습률을 증가시키고 트리의 개수를 늘리면 성능 향상

In [11]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators = 500 , learning_rate = 0.2 , random_state = 42)
scores = cross_validate(gb , train_input , train_target , return_train_score = True , n_jobs = -1)
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9464595437171814
0.8780082549788999


In [12]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9464595437171814
0.8780082549788999


In [13]:
gb.fit(train_input , train_target)
print(gb.feature_importances_)

[0.15872278 0.68011572 0.16116151]


In [14]:
from sklearn.model_selection import GridSearchCV

params = {'n_estimators' : [200,300,400,500,600,700,800],
         'learning_rate' : [0.2,0.3,0.4,0.5,0.6,0.7]}

In [15]:
gs = GridSearchCV(GradientBoostingClassifier(random_state = 42) , params , n_jobs = -1)
gs.fit(train_input , train_target)

GridSearchCV(estimator=GradientBoostingClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [200, 300, 400, 500, 600, 700, 800]})

In [16]:
gs.best_params_

{'learning_rate': 0.4, 'n_estimators': 600}

In [17]:
gb = GradientBoostingClassifier(n_estimators = 600 , learning_rate = 0.4 , random_state = 42)
scores = cross_validate(gb , train_input , train_target , return_train_score = True , n_jobs = -1)
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9769097205950705
0.8826264159324795


# 히스토그램 기반 GradientBoosting
 - Histogram-based Gradient Boosting : 정형 데이터를 다루는 머신러닝 알고리즘에서 가장 인기가 높음
 - 훈련 데이터를 256개의 구간을 나눠 학습시킴
 - 트리의 개수를 지정하는 n_eastimators 파라미터 대신 부스팅 횟수지정하는 max_iter 사용
 - 성능을 향상시키기 위해 max_iter 파라미터를 조정

In [18]:
#버전 확인
import sys
import sklearn
print('Python : ' , sys.version)
print('Pandas : ' , pd.__version__)
print('Numpy : ' , np.__version__)
print('sklearn : ' , sklearn.__version__)
print('matplotlib : ' , mpl.__version__)

Python :  3.8.17 (default, Jul  5 2023, 20:35:33) [MSC v.1916 64 bit (AMD64)]
Pandas :  1.5.3
Numpy :  1.24.3
sklearn :  1.3.0
matplotlib :  3.7.2


In [19]:
# 사이킷런 1.0버전 아래에서는
# from sklearn_experimental import enable_hist_gradient_boosting

from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state = 42)
scores = cross_validate(hgb , train_input , train_target , return_train_score = True , n_jobs = -1)

In [20]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9321723946453317
0.8801241948619236


In [21]:
# feature_importances_가 없음. 별도의 계산 수행
from sklearn.inspection import permutation_importance
# permutation_importance : 반환하는 객체는 반복해서 얻는 특성 중요도
# 평균(importances_mean) , 표준편차(importances_std)를 담고 있다.
hgb.fit(train_input , train_target)
result = permutation_importance(hgb , train_input , train_target , n_repeats = 10 , random_state = 42 , n_jobs = -1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [22]:
result

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]),
 'importances_std': array([0.00382333, 0.00401363, 0.00477012]),
 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
         0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
        [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
         0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
        [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
         0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}

In [23]:
result = permutation_importance(hgb , test_input , test_target , n_repeats = 10 , random_state = 42 , n_jobs = -1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


# XGBoost vs LightGBM

 - 히스토그램 그레이디언트 부스팅 전용 라이브러리 , LightGBM , XGBoost , NGBoost ....
 - 트리 기반의 알고리즘의 앙상블 학습에서 각광받는 알고리즘 중 하나
 - GBM에 기반하고 있지만 , GBM의 단점인 느린 수행시간 , 과적합 규제 등을 해결함
 - 뛰어난 예측 성능
 - GBM 대비 빠른 수행 시간
 - 과적합 규제
 - Tree pruning(트리 가지치기) : 긍정 이득이 없는 분할을 가지치기하여 분할 수를 줄임
 - 자체내장된 교차검증
  - 반복 수행시마다 내부적을 교차검증을 수행하여 최적화된 수행횟수를 가져올 수 있다.
  - 지정된 반보고힛수가 아니라 교차검증을 통해 평가 데이터세트의 평가 값이 최적화되면 반복을 중간에 멈출 수 있는 기능이 있다.
 - 결손값 자체 처리
 - XGBoost는 독자적인 XGBoost 모듈과 사이킷런 프레임워크 기반의 모듈이 존재
 - 독자적인 모듈은 고유의 AP와 하이퍼파라미터를 사용하지만 , 사이킷런 기반 모듈에서는 다른 Estimator와 동일한 사용법을 가짐

In [24]:
import xgboost
print(xgboost.__version__)

1.7.3


In [28]:
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method = 'hist' , random_state = 42)
scores = cross_validate(xgb , train_input , train_target , return_train_score = True)

In [29]:
print(scores['train_score'].mean())
print(scores['test_score'].mean())

0.9555033709953124
0.8799326275264677


In [32]:
pip install lightgbm

  Obtaining dependency information for lightgbm from https://files.pythonhosted.org/packages/b3/f8/ee33e36194eb03a76eccf3adac3fba51f0e56fbd20609bb531659d48d3cb/lightgbm-4.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------  1.3/1.3 MB 42.0 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 27.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier()
cross_validate(lgb , train_input , train_target , force_col_wise=True)

TypeError: got an unexpected keyword argument 'force_col_wise'